# Function approximation

## Taylor approximation
We'll first show how you can use cicada for evaluating function approximations for any callable function in python. We generate coefficients for the approximating polynomials on the fly then use Cicada to evaluate the result of evaluating the approximated function on the secret shared argument. 

The first function we'll demonstrate is for hyperbolic tangent as this is a useful and common activation function for machine learning applications.

In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator

x = numpy.linspace(-5, 5, 100)
y = numpy.tanh(x)

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator)
    
    x_share = protocol.share(src=0, secret=x, shape=x.shape)
    y_hat_share = protocol.taylor_approx(numpy.tanh, x_share, degree=10)
    y_hat = protocol.reveal(y_hat_share)

    return y_hat

results = SocketCommunicator.run(fn=main, world_size=3, show_traceback=True)

#import toyplot
#y_hat = results[0]
#canvas, axes, mark = toyplot.plot(x, y, width=800, height=400)
#axes.plot(x, y_hat);

Comm world player 0 failed: ValueError('setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (11,) + inhomogeneous part.')
Comm world player 1 failed: ValueError('setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (11,) + inhomogeneous part.')
Comm world player 2 failed: ValueError('setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (11,) + inhomogeneous part.')
********************************************************************************
Comm world player 0 traceback:
Traceback (most recent call last):
  File "/Users/tshead/miniconda3/envs/cicada/lib/python3.11/site-packages/cicada/communicator/socket/__init__.py", line 811, in launch
    result = fn(communicator, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folde

As you can see from the mean error, standard deviation, and plot, it's "ok" for operands near the center of the function's approximation range (0, by default) though it is decidedly non-awesome, and diverges to positive and negative infinity when the operand gets too far from the point at which the approximation was centered. This is a common and known behavior of Taylor approximation methods. We have also implemented another approximation method which has better/more stable behavior.

## Padé approximation
Testing for numpy tanh

Clearly the process isn't perfect, but it is a MUCH better approximation when compared with the Taylor series, and has the nice attribute of not diverging to positive or negative infinity

Next we'll show how you can do a similar thing on a user defined function, in this case, we'll define our own sigmoid:


That works about as well as was the case for hyperbolic tangent. This isn't a guaranteed-to-work process for all functions though. It is necessary to do a sanity check on the process prior to just throwing it in cicada because there can be unexpected results. This is due to the limiations of the approximation methods, nothing inherent to their use in cicada. Check the following example for a function that behaves poorly under these approximation methods, relu. 